In [1]:
import os
import joblib
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import calendar
from modules.metrics import rmse
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import (
    Input, LSTM, ConvLSTM2D, Dense,
    Dropout, SpatialDropout2D, MaxPooling2D, BatchNormalization,
    TimeDistributed, LeakyReLU, Flatten
)
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
print(gpus)

1 Physical GPUs, 1 Logical GPUs
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 資料分割

In [3]:
# # split data by month
# def split_data_by_month(data_path):
#     train_files, val_files, test_files = [], [], []
#     file_list = sorted(os.listdir(data_path))  # 20210101-160505_1_160505.csv

#     for file_name in file_list:
#         datetime = file_name.split('-')[0]
#         year = int(datetime[0:4])
#         month = int(datetime[4:6])
#         day = int(datetime[6:8])
#         # 取得當月的天數
#         days_in_month = calendar.monthrange(year, month)[1]
#         part1_end = int(days_in_month * 0.7)
#         part2_end = int(days_in_month * 0.85)

#         file_path = os.path.join(data_path, file_name)
#         if day <= part1_end:
#             train_files.append(file_path)
#         elif day <= part2_end:
#             val_files.append(file_path)
#         else:
#             test_files.append(file_path)

#     return (train_files, val_files, test_files)


# split data by sequence
def split_data_by_sequence(data_path):
    train_files, val_files, test_files = [], [], []
    file_list = sorted(os.listdir(data_path))  # 20210101-160505_1_160505.csv

    train_cases_to_exclude = {'20210116', '20210530', '20210825', '20210722', '20220904'}

    for file_name in file_list:
        datetime = file_name.split('-')[0]

        file_path = os.path.join(data_path, file_name)
        # 加入至測試集
        if any(datetime.startswith(case) for case in train_cases_to_exclude):
            test_files.append(file_path)
            continue # 加入測試集後跳過當前迴圈，為了不將同一檔案加入訓練集或驗證集
        if datetime.startswith(('2021', '202201', '202202', '202203', '202204', '202205', '202206')):
            train_files.append(file_path)
        if datetime.startswith(('202207', '202208', '202209')):
            val_files.append(file_path)
        if datetime.startswith(('202210', '202211', '202212')):
            test_files.append(file_path)

    return (train_files, val_files, test_files)

# mont and sequence data intersect
# def intersect_test_files(data_path):
#     test_files = []
#     file_list = sorted(os.listdir(data_path))  # 20210101-160505_1_160505.csv

#     for file_name in file_list:
#         datetime = file_name.split('-')[0]

#         file_path = os.path.join(data_path, file_name)
        
#         if not datetime.startswith(('202210', '202211', '202212')):
#             continue

#         datetime = file_name.split('-')[0]
#         year = int(datetime[0:4])
#         month = int(datetime[4:6])
#         day = int(datetime[6:8])
#         # 取得當月的天數
#         days_in_month = calendar.monthrange(year, month)[1]
#         part1_end = int(days_in_month * 0.7)
#         part2_end = int(days_in_month * 0.85)
#         file_path = os.path.join(data_path, file_name)

#         if day > part2_end:
#             test_files.append(file_path)

#     return test_files

# 資料縮放器

In [4]:
def fit_scaler_on_training_data(column_name, csv_files):
    """
    預先處理和縮放數據集，將經緯度差值縮放到同一範圍。
    """
    lat_diff_list = []
    lng_diff_list = []

    for csv_file_path in csv_files:
        csv_file = os.path.basename(csv_file_path)

        data = pd.read_csv(csv_file_path, encoding='utf-8', dtype={'fileName': str, 'day': str, 'time': str})

        # 計算緯度和經度的差值
        lat_diff = data[column_name[0]].diff().dropna().reset_index(drop=True)
        lng_diff = data[column_name[1]].diff().dropna().reset_index(drop=True)

        lat_diff_list.extend(lat_diff.values)
        lng_diff_list.extend(lng_diff.values)

    # 將經緯度差值進行縮放
    combined_data = np.vstack([lat_diff_list, lng_diff_list]).T
    scaler = MinMaxScaler()
    scaler.fit(combined_data)

    return scaler

# 資料生成器

In [5]:
def csv_sliding_window_generator(csv_files, column_name, scaler, window_size, step_size, mode):
    """
    預先處理CSV的滑動窗口，並將其生成。
    """

    for csv_file_path in csv_files:
        csv_file = os.path.basename(csv_file_path)
        data = pd.read_csv(csv_file_path, encoding='utf-8',
                           dtype={'fileName': str, 'day': str, 'time': str})
        # data_diff = data[column_name].diff().dropna().reset_index(drop=True)
        
        # 提取 X（經過縮放）和 y（未縮放）
        lat_diff = data[column_name[0]].diff().dropna().reset_index(drop=True)
        lng_diff = data[column_name[1]].diff().dropna().reset_index(drop=True)
        combined_data = np.vstack([lat_diff.values, lng_diff.values]).T  # 經緯度差異合併

        # 使用scaler對X進行縮放
        scaled_data_diff = scaler.transform(combined_data)
        
        x, y_lat, y_lng = [], [], []
        for i in range(0, len(scaled_data_diff) - window_size, step_size):
            x.append(scaled_data_diff[i:i + window_size])  # X是已經經過縮放的數據
            y_lat.append(lat_diff.values[i + window_size])  # y_lat不縮放
            y_lng.append(lng_diff.values[i + window_size])  # y_lng不縮放

        for x_sample, y_lat_sample, y_lng_sample in zip(x, y_lat, y_lng):
            yield np.array(x_sample, dtype=np.float32), {  
                'lstm_lat_output': np.array([y_lat_sample], dtype=np.float32),  # 緯度差標籤不縮放
                'lstm_lng_output': np.array([y_lng_sample], dtype=np.float32)   # 經度差標籤不縮放
            }

def create_csv_sliding_window_dataset(csv_files, column_name, scaler, window_size, step_size, mode):
    """
    使用滑動窗口生成TensorFlow Dataset，避免在生成過程中處理時間序列。
    """
    dataset = tf.data.Dataset.from_generator(
        lambda: csv_sliding_window_generator(csv_files, column_name, scaler, window_size, step_size, mode),
        output_signature=(
            tf.TensorSpec(shape=(window_size, 2), dtype=tf.float32), # X shape: (window_size, 2)
            {
                'lstm_lat_output': tf.TensorSpec(shape=(1,), dtype=tf.float32),  # 緯度標籤
                'lstm_lng_output': tf.TensorSpec(shape=(1,), dtype=tf.float32)   # 經度標籤
            },
        )
        
    )
    return dataset


In [6]:
split_data_mode = 'sequence'  # 'month' or 'sequence' or 'old_dataset'
cells_csv_path = r'H:\cell_data_processed\cells'

if split_data_mode == 'month':
    train_files, val_files, test_files = split_data_by_month(cells_csv_path)
else:
    train_files, val_files, test_files = split_data_by_sequence(cells_csv_path)

# 使用共同測試資料區間
# test_files = intersect_test_files(cells_csv_path)
print('train_files:', len(train_files))
print('val_files:', len(val_files))
print('test_files:', len(test_files))

train_files: 60330
val_files: 10192
test_files: 10251


In [ ]:
column_name = ['Latitude', 'Longitude']
# 設定滑動窗口的參數
window_size = 2  # 窗口大小
step_size = 1  # 步長

# 創建經緯度滑動窗口數據集
# scaler = fit_scaler_on_training_data(column_name, train_files)  # 擬合scaler
# joblib.dump(scaler, f'config/{split_data_mode}/lstm_multitask_scaler.gz')  # 保存scaler
scaler = joblib.load(f'config/{split_data_mode}/lstm_multitask_scaler.gz')  # 載入scaler

# 創建csv數據集
train_dataset = create_csv_sliding_window_dataset(
    train_files, column_name, scaler, window_size, step_size, mode='train')

val_dataset = create_csv_sliding_window_dataset(
    val_files, column_name, scaler, window_size, step_size, mode='val')

In [8]:
print(train_dataset.element_spec)
print(val_dataset.element_spec)

(TensorSpec(shape=(2, 2), dtype=tf.float32, name=None), {'lstm_lat_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'lstm_lng_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None)})
(TensorSpec(shape=(2, 2), dtype=tf.float32, name=None), {'lstm_lat_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'lstm_lng_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None)})


In [9]:
# for x, y in train_dataset.take(1):
#     print(f'X shape: {x.shape}')
#     print(f'Lat y shape: {y["lstm_lat_output"].shape}')
#     print(f'Lng y shape: {y["lstm_lng_output"].shape}')

# for x, y in val_dataset.take(1):
#     print(f'X shape: {x.shape}')
#     print(f'Lat y shape: {y["lstm_lat_output"].shape}')
#     print(f'Lng y shape: {y["lstm_lng_output"].shape}')

# 建立模型

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5,
                               verbose=1, mode='auto', restore_best_weights=True)

checkpoint = ModelCheckpoint(os.path.join(
    os.getcwd(), 'weights', split_data_mode, 'lstm_multitask_0419', 'lstm_mt_diff2-1_e{epoch:02d}v{val_loss:.4f}'),
    monitor='val_loss', save_best_only=True)

# Multi-task LSTM model
n_in = window_size
n_features = 2
inputs = Input(shape=(n_in, n_features), name='lstm_input')

lstm_1 = LSTM(32, activation=LeakyReLU(),
              return_sequences=True, name='lstm_1')(inputs)
lstm_1 = BatchNormalization(name='lstm_1-bn_1')(lstm_1)
dropout_1 = Dropout(0.2, name='lstm_1-dropout_1')(lstm_1)

lstm_2 = LSTM(64, activation=LeakyReLU(),
              return_sequences=True, name='lstm_2')(dropout_1)
lstm_2 = BatchNormalization(name='lstm_2-bn_2')(lstm_2)
dropout_2 = Dropout(0.2, name='lstm_2-dropout_2')(lstm_2)

lstm_3 = LSTM(128, activation=LeakyReLU(),
              return_sequences=False, name='lstm_3')(dropout_2)
lstm_3 = BatchNormalization(name='lstm_3-bn_3')(lstm_3)
dropout_3 = Dropout(0.2, name='lstm_3-dropout_3')(lstm_3)

dense_1 = Dense(64, kernel_regularizer=l2(
    0.01), activation='linear', name='lstm_dense')(dropout_3)
dropout_4 = Dropout(0.2, name='lstm_dense-dropout')(dense_1)

lat_output = Dense(1, activation='linear', name='lstm_lat_output')(dropout_4)
lng_output = Dense(1, activation='linear', name='lstm_lng_output')(dropout_4)

lstm_model = Model(inputs=inputs, outputs=[
                   lat_output, lng_output], name='lstm_multi_task')

lstm_model.compile(optimizer=Adam(learning_rate=0.0001),
                   loss={'lstm_lat_output': 'mse',
                         'lstm_lng_output': 'mse'},
                   loss_weights={'lstm_lat_output': 1.0,
                                 'lstm_lng_output': 1.0},
                   metrics={'lstm_lat_output': ['mse', rmse, 'mae'],
                            'lstm_lng_output': ['mse', rmse, 'mae']})

print(lstm_model.summary())

Model: "lstm_multi_task"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lstm_input (InputLayer)        [(None, 2, 2)]       0           []                               
                                                                                                  
 lstm_1 (LSTM)                  (None, 2, 32)        4480        ['lstm_input[0][0]']             
                                                                                                  
 lstm_1-bn_1 (BatchNormalizatio  (None, 2, 32)       128         ['lstm_1[0][0]']                 
 n)                                                                                               
                                                                                                  
 lstm_1-dropout_1 (Dropout)     (None, 2, 32)        0           ['lstm_1-bn_1[0][0]

In [9]:
# 設定訓練參數
batch_size = 32
epochs = 50

# 使用 .batch() 和 .prefetch() 進行數據集的優化加載
train_dataset = train_dataset.batch(batch_size)\
                .prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = val_dataset.batch(batch_size)\
                .prefetch(tf.data.experimental.AUTOTUNE)

In [10]:
# 開始訓練模型
history = lstm_model.fit(
    train_dataset,
    validation_data=val_dataset,  # 傳入驗證集
    epochs=epochs,
    callbacks=[early_stopping, checkpoint],
    verbose=1  # 訓練過程中打印進度
)

NameError: name 'lstm_model' is not defined

# 評估模型

In [11]:
column_name = ['Latitude', 'Longitude']
# 設定滑動窗口的參數
window_size = 2  # 窗口大小
step_size = 1  # 步長

# 創建csv數據集
test_dataset = create_csv_sliding_window_dataset(
    test_files, column_name, scaler, window_size, step_size, mode='test')

In [12]:
test_dataset.element_spec

(TensorSpec(shape=(2, 2), dtype=tf.float32, name=None),
 {'lstm_lat_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None),
  'lstm_lng_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None)})

In [13]:
# 設定訓練參數
batch_size = 32
epochs = 50

# 使用 .batch() 和 .prefetch() 進行數據集的優化加載
test_dataset = test_dataset.batch(batch_size)\
                .prefetch(tf.data.experimental.AUTOTUNE)

In [14]:
if split_data_mode == 'month':
    # model_path = os.path.join(os.getcwd(), r'weights\month\lstm_multitask\')
    pass
elif split_data_mode == 'sequence':
    model_path = os.path.join(os.getcwd(), r'weights/sequence/lstm_multitask_0419/lstm_mt_diff2-1_e08v0.0010')
    
if os.path.exists(model_path):
    lstm_model = load_model(model_path, custom_objects={'rmse': rmse})
    print('Load model successfully!')
    print(lstm_model.summary())

Load model successfully!
Model: "lstm_multi_task"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lstm_input (InputLayer)        [(None, 2, 2)]       0           []                               
                                                                                                  
 lstm_1 (LSTM)                  (None, 2, 32)        4480        ['lstm_input[0][0]']             
                                                                                                  
 lstm_1-bn_1 (BatchNormalizatio  (None, 2, 32)       128         ['lstm_1[0][0]']                 
 n)                                                                                               
                                                                                                  
 lstm_1-dropout_1 (Dropout)     (None, 2, 32)        0     

In [ ]:
from haversine import haversine

distances = []
lat_errors = []
lon_errors = []

for csv_file in test_files:
    df = pd.read_csv(csv_file)

    # 真實經緯度序列
    lats = df['Latitude'].values
    lons = df['Longitude'].values

    # 差分計算
    delta_lat = np.diff(lats)
    delta_lon = np.diff(lons)
    combined_data = np.vstack([delta_lat, delta_lon]).T

    # 特徵縮放
    scaled_data_diff = scaler.transform(combined_data)

    # 準備滑動窗口資料
    inputs = []
    for i in range(len(scaled_data_diff) - window_size):
        input_sample = [scaled_data_diff[i], scaled_data_diff[i + 1]]
        inputs.append(input_sample)

    inputs = np.array(inputs)

    # 模型預測差值
    pred_lats_diff, pred_lons_diff = lstm_model.predict(inputs)

    # 差值還原回經緯度
    pred_lats = []
    pred_lons = []
    for i, (dlat, dlon) in enumerate(zip(pred_lats_diff, pred_lons_diff)):
        base_lat = lats[i + 2]
        base_lon = lons[i + 2]
        pred_lat = base_lat + dlat.item()
        pred_lon = base_lon + dlon.item()
        pred_lats.append(pred_lat)
        pred_lons.append(pred_lon)

    # 計算實際誤差
    for i in range(len(pred_lats)):
        real_lat = lats[i + 3]
        real_lon = lons[i + 3]
        pred_lat = pred_lats[i]
        pred_lon = pred_lons[i]

        lat_errors.append(real_lat - pred_lat)
        lon_errors.append(real_lon - pred_lon)
        distances.append(haversine((real_lat, real_lon), (pred_lat, pred_lon)))

# === 評估指標 ===
lat_errors = np.array(lat_errors)
lon_errors = np.array(lon_errors)
distances = np.array(distances)

# MAE
lat_mae = np.mean(np.abs(lat_errors))
lon_mae = np.mean(np.abs(lon_errors))

# MSE
lat_mse = np.mean(lat_errors ** 2)
lon_mse = np.mean(lon_errors ** 2)

# RMSE
lat_rmse = np.sqrt(lat_mse)
lon_rmse = np.sqrt(lon_mse)

# 結果輸出
print("=== 經緯度位置評估 ===")
print(f"Latitude MAE: {lat_mae:.6f} 度")
print(f"Longitude MAE: {lon_mae:.6f} 度")
print(f"Latitude MSE: {lat_mse:.6f}")
print(f"Longitude MSE: {lon_mse:.6f}")
print(f"Latitude RMSE: {lat_rmse:.6f}")
print(f"Longitude RMSE: {lon_rmse:.6f}")
print(f"Average Haversine distance: {np.mean(distances):.6f} km")


1/1 [==============================] - 0s 17ms/step
=== 經緯度位置評估 ===
Latitude MAE: 0.014968 度
Longitude MAE: 0.018340 度
Latitude MSE: 0.000393
Longitude MSE: 0.000537
Latitude RMSE: 0.019828
Longitude RMSE: 0.023164
Average Haversine distance: 2.83 km


In [16]:
# 結果輸出
print("=== 經緯度位置評估 ===")
print(f"Latitude MAE: {lat_mae:.6f} 度")
print(f"Longitude MAE: {lon_mae:.6f} 度")
print(f"Latitude MSE: {lat_mse:.6f}")
print(f"Longitude MSE: {lon_mse:.6f}")
print(f"Latitude RMSE: {lat_rmse:.6f}")
print(f"Longitude RMSE: {lon_rmse:.6f}")
print(f"Average Haversine distance: {np.mean(distances):.6f} km")

=== 經緯度位置評估 ===
Latitude MAE: 0.014968 度
Longitude MAE: 0.018340 度
Latitude MSE: 0.000393
Longitude MSE: 0.000537
Latitude RMSE: 0.019828
Longitude RMSE: 0.023164
Average Haversine distance: 2.825186 km


In [ ]:
# from haversine import haversine

# distances = []
# lat_errors = []
# lon_errors = []

# for csv_file in test_files:
#     df = pd.read_csv(csv_file)
    
#     # 取得經緯度真實值
#     lats = df['Latitude'].values
#     lons = df['Longitude'].values

#     # 計算差值 (差分)
#     delta_lat = np.diff(lats)
#     delta_lon = np.diff(lons)

#     combined_data = np.vstack([delta_lat, delta_lon]).T  # 經緯度差異合併

#     # 使用scaler對X進行縮放
#     scaled_data_diff = scaler.transform(combined_data)

#     # print(scaled_data_diff)
#     # print('*' * 20)
#     # print(f'lats: {lats}')
#     # print(f'lons: {lons}')
#     # print(f'delta_lat: {delta_lat}')
#     # print(f'delta_lon: {delta_lon}')

#     # 準備滑動窗口資料（輸入兩步差值預測一步）
#     inputs = []
#     targets = []
#     for i in range(len(scaled_data_diff) - 2):
#         # print(scaled_data_diff)
#         # print(scaled_data_diff[i:i+2])
#         input_sample = [scaled_data_diff[i], scaled_data_diff[i+1]]
#         # target_sample = combined_data[i+2]
#         inputs.append(input_sample)
#         targets.append(target_sample)

#     inputs = np.array(inputs)  # shape: (samples, 2, 2)
#     # targets = np.array(targets)
#     # print(f'inputs shape: {inputs.shape}')  # (samples, 2, 2)
#     # print(f'targets shape: {targets.shape}')  # (samples, 2)

#     # print(f'inputs: {inputs}')
#     # print(f'targets: {targets}')

#     # 模型預測
#     pred_lats_diff, pred_lons_diff = lstm_model.predict(inputs)  # shape: (samples, 2)

#     # print(f'pred_lats_diff: {pred_lats_diff}') # [[0.00868985]]
#     # print(f'pred_lons_diff: {pred_lons_diff}') # [[-0.07842029]]
#     # 預測差值還原回經緯度
#     pred_lats = []
#     pred_lons = []
#     for i, pred in enumerate(zip(pred_lats_diff, pred_lons_diff)):
#         last_lat = lats[i+2]  # 這一步你可能會根據實際資料調整
#         last_lon = lons[i+2]
#         pred_lat = last_lat + pred[0].item()
#         pred_lon = last_lon + pred[1].item()
#         pred_lats.append(pred_lat)
#         pred_lons.append(pred_lon)

#     # 計算預測與實際點的距離
#     for i in range(len(pred_lats)):
#         # print(f'預測經緯度: {pred_lats[i]}, {pred_lons[i]}')
#         # print(f'實際經緯度: {lats[i + 3]}, {lons[i + 3]}')
#         real_lat = lats[i + 3]  # 因為你預測的是 i+2 的下一步（i+3）
#         real_lon = lons[i + 3]
#         error_lat = abs(real_lat - pred_lats[i])
#         error_lon = abs(real_lon - pred_lons[i])
#         lat_errors.append(error_lat)
#         lon_errors.append(error_lon)
#         dist = haversine((real_lat, real_lon), (pred_lats[i], pred_lons[i]))
#         distances.append(dist)

# avg_distance = np.mean(distances)
# print(f"平均誤差距離：{avg_distance:.2f} km")

# avg_lat_error = np.mean(lat_errors)
# avg_lon_error = np.mean(lon_errors)
# print(f"平均經度差：{avg_lon_error:.6f} 度")
# print(f"平均緯度差：{avg_lat_error:.6f} 度")

# print(f"總經度差：{sum(lon_errors):.6f} 度")
# print(f"總緯度差：{sum(lat_errors):.6f} 度")